In [5]:
#importing computer vision and numpy
import cv2
import numpy as np
import mediapipe as mp
from math import hypot
import screen_brightness_control as sbc
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
import time

In [6]:
def main():
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volRange = volume.GetVolumeRange()
    minVol, maxVol, _ = volRange

    mpHands = mp.solutions.hands
    hands = mpHands.Hands(static_image_mode=False, max_num_hands=2, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    draw = mp.solutions.drawing_utils
    
    cap = cv2.VideoCapture(0)
    
    frame_count = 0
    detection_count = 0
    brightness_values = []  # For stability
    distance_values = []   # For precision
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_count += 1
            
            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)
            
            if processed.multi_hand_landmarks:
                detection_count += 1
                print(f"Hands detected! Frame: {frame_count}, Detection: {detection_count}")
            else:
                print(f"No hands detected. Frame: {frame_count}, Detection: {detection_count}")
            
            left_landmark_list, right_landmark_list = get_left_right_landmarks(frame, processed, draw, mpHands)

            # Brightness control with latency, stability, and precision
            if left_landmark_list:
                start_time = time.time()
                left_distance = get_distance(frame, left_landmark_list)
                b_level = np.interp(left_distance, [50,220], [0,100])
                sbc.set_brightness(b_level)
                end_time = time.time()
                latency_ms = (end_time - start_time) * 1000
                print(f"Brightness Latency: {latency_ms:.2f} ms")
                
                # Stability
                brightness_values.append(b_level)
                if len(brightness_values) > 100:
                    stability = np.std(brightness_values[-100:])
                    print(f"Brightness Stability (std dev): {stability:.2f}%")
                
                # Precision
                distance_values.append(left_distance)
                if len(distance_values) > 100:
                    avg_distance = np.mean(distance_values[-100:])
                    distance_std = np.std(distance_values[-100:])
                    print(f"Precision - Avg Distance: {avg_distance:.2f} px, Std Dev: {distance_std:.2f} px")
            
            # Volume control
            if right_landmark_list:
                right_distance = get_distance(frame, right_landmark_list)
                vol = np.interp(right_distance, [50,220], [minVol,maxVol])
                volume.SetMasterVolumeLevel(vol, None)
            
            # Detection accuracy: Print every 10 frames instead of 30
            accuracy = (detection_count / frame_count * 100) if frame_count > 0 else 0
            if frame_count % 10 == 0:  # Changed from 30 to 10
                print(f"Detection Accuracy: {accuracy:.2f}% (Frames: {frame_count}, Detections: {detection_count})")
            
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == 27:
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

def get_left_right_landmarks(frame, processed, draw, hands):
    left_landmarks_list = []
    right_landmarks_list = []

    if processed.multi_hand_landmarks:
        for handlm in processed.multi_hand_landmarks:
            for idx, found_landmark in enumerate(handlm.landmark):
                height, width, _ = frame.shape
                x, y = int(found_landmark.x*width), int(found_landmark.y*height)

                if idx == 4 or idx == 8:
                    landmark = [idx, x, y]
                    if handlm == processed.multi_hand_landmarks[0]:
                        left_landmarks_list.append(landmark)
                    elif handlm == processed.multi_hand_landmarks[1]:
                        right_landmarks_list.append(landmark)
            draw.draw_landmarks(frame, handlm, hands.HAND_CONNECTIONS)
    
    return left_landmarks_list, right_landmarks_list

def get_distance(frame, landmark_list):
    if len(landmark_list) < 2:
        return 

    (x1, y1), (x2, y2) = (landmark_list[0][1], landmark_list[0][2]), \
        (landmark_list[1][1], landmark_list[1][2])

    cv2.circle(frame, (x1, y1), 7, (0,255,0), cv2.FILLED)
    cv2.circle(frame, (x2, y2), 7, (0,255,0), cv2.FILLED)
    cv2.line(frame, (x1, y1), (x2, y2), (0,255,0), 3)

    L = hypot(x2-x1, y2-y1)
    return L

In [7]:
if __name__ == '__main__':
    main();

No hands detected. Frame: 1, Detection: 0
No hands detected. Frame: 2, Detection: 0
No hands detected. Frame: 3, Detection: 0
No hands detected. Frame: 4, Detection: 0
No hands detected. Frame: 5, Detection: 0
No hands detected. Frame: 6, Detection: 0
No hands detected. Frame: 7, Detection: 0
No hands detected. Frame: 8, Detection: 0
No hands detected. Frame: 9, Detection: 0
No hands detected. Frame: 10, Detection: 0
Detection Accuracy: 0.00% (Frames: 10, Detections: 0)
No hands detected. Frame: 11, Detection: 0
No hands detected. Frame: 12, Detection: 0
No hands detected. Frame: 13, Detection: 0
No hands detected. Frame: 14, Detection: 0
No hands detected. Frame: 15, Detection: 0
No hands detected. Frame: 16, Detection: 0
No hands detected. Frame: 17, Detection: 0
No hands detected. Frame: 18, Detection: 0
No hands detected. Frame: 19, Detection: 0
No hands detected. Frame: 20, Detection: 0
Detection Accuracy: 0.00% (Frames: 20, Detections: 0)
No hands detected. Frame: 21, Detection: 